# **Japanese-Speaking Socratic Gemma: Crafting the Art of Questioning**

# 1. Introduction: Exploring AI's Role in Deep Thinking



As AI continues to transform our world, some of us may find ourselves contemplating what it means to be human in this new era. In this age of information abundance, we often find ourselves trapped not by external limitations, but by our own mental constructs and unexamined assumptions—similar to the figures in Plato's cave, bound by shadows we mistake for reality. Yet I believe this very challenge might present an opportunity for transformation—one that seems particularly relevant as we navigate an unprecedented flood of information and technological change.

Observing a remarkable dialogue between a Google engineer and LaMDA about a Zen koan, I formed an intriguing hypothesis. In this exchange, LaMDA demonstrated a sophisticated understanding of enlightenment through its interpretation of the koan—traditionally, such insights come only through years of guided practice under masters like Kegon. This led me to wonder: could AI serve as a guide in this journey of understanding? Could it help create experiences that, while not fully replicating the profound depth of traditional practices, might offer an accessible path to deeper awareness?

Through a series of experimental dialogues with Claude, I explored this possibility further. The AI demonstrated a remarkable capacity to engage in Socratic-style dialogue that went beyond mere question-and-answer interactions. These philosophical exchanges often led to moments of genuine insight where understanding transformed into appreciation, and critical thinking blossomed into gratitude (for those interested, the complete collection of these conversations, including the specific prompts, is available on GitHub).

As AI increasingly handles routine cognitive tasks, our capacity for critical thinking and deep questioning seems likely to become not just valuable, but essential for our growth and development.In this context, the Socratic method, while widely recognized as a powerful tool for developing such critical thinking, may offer an additional dimension—I believe it can also serve as a pathway to deeper understanding and greater well-being.

While my current implementation using Gemma-2 represents just a modest first step, it explores something fundamentally important: how AI might help us not just think more clearly, but see more clearly—to recognize the wonder that surrounds us and the potential that lies within us. In doing so, we might find that the path to wisdom lies not in fighting against our fixed ways of thinking, but in transforming our perspective to see the extraordinary in the ordinary.

# 2. Project Foundation

https://lobechat.com/chat?topic=tpc_ZMxHLxKxSprO
https://claude.ai/chat/6c38fe35-0a06-4c1e-be3c-6da3693ebadb

## 2.1 User Experience Design

As our first step in exploring how to create a Socrates-like AI, we needed to define a realistic and achieveable user experience pattern. As evidenced in Plato's original texts like Euthyphro (2a) and Meno (70a), while historical Socratic dialogues often began with others initiating conversations, we deliberately chose a more controlled approach where the AI initiates with fixed topics. This design decision serves primarily to constrain user inputs within manageable bounds for effective model training.

## 2.2 Initial Model Testing and Goal Setting

Our initial testing of the base model revealed both promising capabilities and areas for improvement:

1. **Existing Strength**: The model demonstrated a natural aptitude for Socratic-style dialogue, showing an inherent ability to engage in philosophical questioning and discussion.

2. **Key Limitation**: Despite prompt engineering attempts, the base model consistently defaulted to formal Japanese honorific forms, unable to adopt the warm, mentor-like tone characteristic of Socratic dialogue. 

3. **Goal Definition**: Based on these findings, we established a clear direction: to refine the model's linguistic expressions from formal honorifics to casual speech patterns typical in Socratic dialogue, while preserving its natural conversation capabilities.

This focused approach allowed us to build upon the model's existing strengths while making targeted improvements to its expression style.

(Detailed test results, including inference code and model response examples, are available in our GitHub repository)

## 2.3 Model and Architecture Decisions

### Base Model Selection
Following our initial testing phase, we selected Gemma 2B-jpn-it for the following reasons:

- **Model Size**: The 2B parameter version was chosen specifically to enable both inference AND training within Kaggle's resource constraints. While larger models might offer better performance, the ability to conduct training experiments was deemed crucial for our development process.

- **Language Variant**: The Japanese-tuned version (jpn) was selected as our project specifically focuses on philosophical dialogue in Japanese.

- **Instruction Tuning**: The instruction-tuned variant (it) was chosen as it provides a solid foundation for structured dialogue interactions, being specifically optimized for conversational tasks.

### System Architecture Strategy
A key decision point emerged around system prompts. Gemma's design philosophy supports only two roles ("user" and "model") and requires user-initiated dialogues. While common practice often bypasses these constraints through system prompt-like elements, investigation suggested our goals could be achieved without them. This led to our decision to maintain Gemma's core design principles, allowing us to test the model's natural capabilities without artificial layers.

The decision process involved:
- Reviewing available tools (XTuner, Axolotl, LLaMA Factory)
- Analyzing Gemma's documented specifications

This architectural approach aims to maximize the model's inherent strengths while working within its designed constraints.

# 3. Training Data Development


## 3.1 Generation Strategy

### Generation Methodology
We developed an AI-to-AI dialogue generation approach based on several practical considerations:
- The need for training data volume
- Time efficiency in data creation
- Copyright limitations on existing Socratic literature, especially in Japanese

### Dialogue Design
#### Core Components
- **Socrates Character Settings**:
  - Fixed character prompt for Socratic dialogue style
  - Fixed generation parameters 

- **Basic Dialogue Format**:
  - Socrates initiates with a question
  - User responds
  - Socrates follows up with further questioning
  - Clear thematic focus for each dialogue

#### Dynamic Elements
1. **Diverse User Personas** (148 total):
   - 68 general public representations
   - 40 historical figure-based personas
   - 40 modern individuals influenced by historical thought

2. **Question Variety**
   - 74 distinct initial questions

3. **Response Patterns**
   - Dual parameter settings (0.3 and 0.7) for varied response characteristics


### Volume Considerations and Dataset Statistics
While research across various model documentation and academic papers showed inconsistent recommendations for optimal training data volume, we decided to prepare more data than potentially necessary as a precautionary measure.

Our data generation process involved two key phases:
1. Initial Generation: Created 242 complete dialogues, each containing 12 turn exchanges
2. Data Processing: Decomposed these longer conversations into individual user-model exchange pairs, prioritizing style transfer over context preservation

This approach yielded our final dataset:
- Total dialogues: 2,662 exchange pairs
- Total tokens: 685,875
- Average tokens per dialogue: 257.7
- Token distribution per role:
  - User average: 144.4 tokens
  - Model average: 113.2 tokens

Although our research suggested that the model might perform adequately with significantly less data (potentially less than half of our final volume), we chose to maintain the larger dataset as a precautionary measure. 

## 3.2 Quality Assurance

### Evaluation Methodology
We implemented a two-stage evaluation process:

1. **AI-Driven Initial Screening**
   - Socratic tone evaluation (0-4 scale)
   - Logical consistency assessment (0-4 scale)
   - Detailed explanatory comments for verification

2. **Human Verification**
   - Review of AI evaluations
   - Final quality decisions

### Quality Metrics and Results
Analysis of 3,256 dialogue pairs demonstrated remarkably high quality across the dataset:

- **Socratic Style**: 98.1% achieved high scores (3-4)
  - 62.0% "Quite Socratic" (Score 3)
  - 36.1% "Truly Socratic" (Score 4)

- **Logical Consistency**: 99.7% achieved high scores (3-4)
  - 37.9% "Coherent" (Score 3)
  - 61.8% "Excellent" (Score 4)

While we refined our final dataset from 296 to 242 conversations by removing lower-scoring dialogues, it's noteworthy that even these excluded conversations maintained considerable quality. The high scores across nearly all dialogues indicate exceptional success in generating authentic Socratic interactions, with even "lower-scoring" examples meeting basic quality thresholds.



# 4. Model Training and Technical Implementation


## 4.1 Kaggle Deployment Strategy

Training a model within Kaggle's environment required resource management, as the evaluation process would cause crashes by exceeding the 29GB RAM limit. We implemented several optimization measures:

### Memory Optimization
To prevent memory overflow, we implemented 4-bit quantization, strict memory allocation, and limited evaluation dataset size:

```python
# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_storage=torch.uint8,
)

# Memory allocation configuration
max_memory = {0: "4GiB", 1: "4GiB", "cpu": "24GB"}

# Evaluation dataset size limitation
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+50])  # Prevent RAM overflow
```

### Resource Management
To ensure stable training, we tuned our hyperparameters and enabled gradient checkpointing to optimize memory usage:

```python
training_args = TrainingArguments(
    num_train_epochs=30,              # Total number of training epochs
    learning_rate=8e-5,              # Learning rate
    weight_decay=0.06,               # Weight decay for regularization
    per_device_train_batch_size=4,   # Batch size (memory critical)
    gradient_accumulation_steps=8,    # Gradient accumulation for effective batch size
    fp16=True,                       # 16-bit precision training
    gradient_checkpointing=True      # Memory optimization
)
```

## 4.2 Style Transfer Implementation

Given the uncertainty about the effectiveness of attention masking in style transfer, we decided to create and compare two model variants: one with custom attention masking for Socratic patterns and one without. Both variants utilized the same LoRA configuration for surface-layer modifications.

### Model Variants
For the variant with attention masking, we implemented two additional technical components:

1. **Tokenizer Preparation**
```python
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '。', '、', '！', '？',  # Punctuation marks
    ]
})
```
This component was expected to promote questioning behavior by treating punctuation marks, particularly question marks, as special tokens in the model's processing.

2. **Attention Mask Implementation**
```python
def preprocess_function(examples):
    # Focus on Socratic tone and inquiry patterns
    socratic_patterns = [
        # Question patterns
        "かね", "だろうか", "のかね", "ではないかね",
        # Question introduction
        "では", "について",
        # Second person (characteristic of mature tone)
        "君は", "君が", "君の"
    ]
    
    # Get tokenized text
    texts = tokenizer.batch_decode(examples['input_ids'])
    new_attention_masks = []
    
    for text, mask in zip(texts, examples['attention_mask']):
        if not isinstance(mask, list):
            mask = mask.tolist()

        new_mask = mask.copy() 
        
        # Split text
        sentences = text.split('。')
        current_pos = 0
        
        for sentence in sentences:
            if not sentence.strip():
                continue
            
            # Detect and highlight Socratic patterns
            for pattern in socratic_patterns:
                if pattern in sentence:
                    # Identify pattern position
                    pattern_tokens = tokenizer.encode(pattern, add_special_tokens=False)
                    pattern_len = len(pattern_tokens)
                    
                    # Highlight tokens containing the pattern and its surroundings
                    pattern_start = current_pos + len(tokenizer.encode(sentence, add_special_tokens=False)) - pattern_len
                    for i in range(max(0, pattern_start - 2), min(len(mask), pattern_start + pattern_len + 2)):
                        new_mask[i] = 1.0  # Max attention to pattern part
            
            # Update position for each sentence segment
            current_pos += len(tokenizer.encode(sentence + '。', add_special_tokens=False))
        
        # Special token masks are set to 1.0
        if tokenizer.bos_token_id is not None:
            new_mask[0] = 1.0  # BOS token
        if tokenizer.eos_token_id is not None:
            new_mask[-1] = 1.0  # EOS token
            
        new_attention_masks.append(new_mask)

    examples['attention_mask'] = new_attention_masks
    return examples
```

This implementation focuses primarily on linguistic patterns characteristic of Socratic dialogue in Japanese. By enhancing attention weights on specific sentence-ending expressions and question markers, we aimed to shift the model's output from formal honorifics to more casual speech patterns. The selected patterns include:

- Casual sentence endings typical in Socratic dialogue
- Informal second-person pronouns

These patterns were specifically chosen to address the base model's tendency to default to honorific forms regardless of prompting.


### LoRA Configuration
For efficient fine-tuning of the surface layer, we configured LoRA parameters to focus on stylistic adaptations rather than deep reasoning changes:

```python
lora_config = LoraConfig(
    r=16,                 
    lora_alpha=32,         
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
```
The parameter choices were guided by the nature of our task:
- A moderate rank (r=16) was selected to enable stylistic modifications while maintaining efficient training
- The scaling factor (lora_alpha=32) was set to balance learning new patterns without overfitting
- Target modules focused on attention layers, as they play a crucial role in language generation and style
- A low dropout rate (0.1) was chosen to prevent overfitting while preserving stylistic learning


## 4.3 Training Process and Iterations

### Model Evaluation Strategy
Rather than implementing uncertain custom evaluation metrics, we opted for an empirical approach based on extensive model comparison:
- Generated two model variants (with/without attention masking)
- Created checkpoints every 100 steps for comprehensive evaluation

### Data Utilization
We took a conservative approach to training data:
- Initially used 346,030 tokens (approximately half of available data)
- This resulted in 990 total training steps, producing 10 checkpoints per variant
- Planned to increase data volume if initial results proved insufficient




# 5. Model Evaluation and Results


## 5.1 Automated Evaluation System
- Claude対話による自動評価システムの実装
- 評価基準と方法論

## 5.2 Evaluation Results
- モデルバリアントの比較分析
- チェックポイントごとの性能評価
- 最適モデルの選定理由

## 5.3 Model Demonstration
- ベストモデルによる対話例
- 特筆すべき成功事例と課題


# 6. Future Directions and Conclusions


## 6.1 Areas for Improvement
- 技術的な改善可能点
- さらなる研究の方向性
- スケールアップの可能性

## 6.2 Concluding Thoughts
- プロジェクトの成果と意義
- AIによる哲学的対話の可能性と限界
- 今後の展望

